# QAOA rešavač linearnih jednačina

Videli smo da je jedan od problema algoritma $AQC(p)$ što zahteva veliki broj kvantnih kola. Ovde ćemo pokazati algoritam koji može da poboljša ovo tako što će sa relativno malo kola da se približi rešenju linearne jednačine.

In [1]:
import scipy.sparse as sparse
import numpy as np
from math import sqrt
from scipy.optimize import minimize
from qiskit.circuit.library import QAOAAnsatz
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import StatePreparation
from qiskit.circuit import QuantumCircuit
from math import log2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import Session, EstimatorV2 as Estimator
from qiskit_ibm_runtime import SamplerV2 as Sampler

rows, cols = 4, 4
density = 0.5

R = sparse.random(rows, cols, density=density, format='csr', dtype=np.float64)
A = R.T @ R + sparse.diags(np.ones(cols) * 10, format='csr')
A = A.toarray()

b = np.random.rand(cols)
c = sqrt(sum(b*b))
b = 1/c * b

X = np.array([[0.0, 1.0], [1.0, 0.0]])
Y = np.array([[0.0, -1.0j], [1.0j, 0.0]])
Z = np.array([[1.0, 0.0], [0.0, -1.0]])

Op, Om = 1/2.0 * (X + 1.0j * Y), 1/2.0 * (X - 1.0j * Y)

A1 = np.kron(Op, A) + np.kron(Om, A.conj().T)
b1 = np.kron(np.array([0, 1]), b)
N = 2*cols

plus = 1/sqrt(2.0) * np.array([1, 1])
Plus = plus.reshape(-1, 1) @ plus.reshape(1, 2)
minus = 1/sqrt(2.0) * np.array([1, -1])
B1 = b1.reshape(-1, 1) @ b1.reshape(1, N)

Qb = np.eye(2*N) - np.kron(Plus, B1)
H0 = np.kron(Op, np.kron(Z, np.eye(N)) @ Qb) + np.kron(Om, Qb @ np.kron(Z, np.eye(N)))
H1 = np.kron(Op, np.kron(X, A1) @ Qb) + np.kron(Om, Qb @ np.kron(X, A1))

Kao i u $AQC(p)$, generisali smo matricu $A$, vektor $b$, transformisali matricu u ermitsku matricu $A1$, transformisali vektor $\ket{b}$ u $\ket{1}\ket{b}$ i izgradili smo Hamiltonijane $H_0$ i $H_1$. Sada preskačemo određivanje kondicione vrednosti i konstantni $T$ i $M$ iz algoritma $AQC(p)$, već odmah prelazimo na konstrukciju kvantnog kola gde je $M = 2$. To kvantno kolo će biti oblika:
\begin{align*}
    e^{-i \gamma_M H_1} e^{-i \beta_M H_0} \ldots e^{-i \gamma_1 H_1} e^{-i \beta_1 H_0} \ket{\psi_0},
\end{align*} gde su parametri $\gamma_i$, $\beta_i$ neke konstante koje ćemo odrediti kasnije.

In [2]:
n = int(log2(H0.shape[0]))

prepare_b = QuantumCircuit(n)
initial_state = np.kron(np.array([1, 0]), np.kron(minus, b1))
stateprep = StatePreparation(initial_state)
prepare_b.append(stateprep, range(n))

h0, h1 = SparsePauliOp.from_operator(H0), SparsePauliOp.from_operator(H1)

M = 2

circuit = QAOAAnsatz(cost_operator = h1, initial_state = prepare_b, mixer_operator = h0, reps = M)

**Qiskit** klasa **QAOAAnsatz** radi sav ovaj posao za nas. Naše je bilo samo da zadamo dubinu kola ($20$), Hamiltonijane $H_0$ i $H_1$. Ovde je $H_1$ **cost_operator**, a $H_0$ **mixer_operator**. Takođe, inicijalizovali smo kvantno kolo koje priprema stanje $\ket{-}\ket{0}\ket{b'}$ i dodali ga na početak kvantnog kola.

Sad je potrebno naći optimalne vrednosti parametara, odnosno one koji minimizuju $\bra{\psi(\gamma, \beta)} H_1^2 \ket{\psi(\gamma, \beta)}$.

In [3]:
h1_squared = h1 @ h1

In [4]:
service = QiskitRuntimeService(channel = 'local')
backend = service.least_busy()

# Create pass manager for transpilation
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
 
candidate_circuit = pm.run(circuit)

In [5]:
initial_gamma = np.pi
initial_beta = np.pi / 2

init_params = [initial_beta for i in range(M)] + [initial_gamma for i in range(M)]

Definišemo funkciju cene. Funkcija je slična onoj koju smo koristili za VQE.

In [6]:
hamiltonian_isa = h1.apply_layout(candidate_circuit.layout)

def cost_func_estimator(params, ansatz, hamiltonian, estimator):
 
    pub = (ansatz, hamiltonian, params)
    job = estimator.run([pub])
 
    results = job.result()[0]
    cost = results.data.evs
 
    objective_func_vals.append(cost)
 
    return cost

Sad sve što treba uraditi je da se proslede odgovarajući argumenti optimizatoru i da se zadatak izvrši.

In [7]:
objective_func_vals = []

with Session(backend=backend) as session:
    
    estimator = Estimator(mode=session)
    estimator.options.default_shots = 100
 
    res = minimize(
        cost_func_estimator,
        init_params,
        args = (candidate_circuit, hamiltonian_isa, estimator),
        method = 'cobyla',
        options = {'maxiter' : 200, 'disp' : 1}
    )


   Normal return from subroutine COBYLA

   NFVALS =   43   F = 8.259875E-01    MAXCV = 0.000000E+00
   X = 1.427210E+00   1.178988E+00   2.785751E+00   2.305426E+00


In [8]:
res

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 0.8259875302886626
       x: [ 1.427e+00  1.179e+00  2.786e+00  2.305e+00]
    nfev: 43
   maxcv: 0.0

Uspelo je da iskonvergira. Sada je potrebno parametre koje smo dobili da ubacimo u ansatz, a zatim izvučemo $\ket{0}\ket{+}\ket{x''}$.

In [12]:
circuit.measure_all()
optimized_circuit = pm.run(circuit)

sampler = Sampler(mode=backend)
sampler.options.default_shots = 10000
 
pub = (optimized_circuit, res.x)
job = sampler.run([pub])
counts_int = job.result()[0].data.meas.get_int_counts()
counts_bin = job.result()[0].data.meas.get_counts()
shots = sum(counts_int.values())
final_distribution_int = {key: val / shots for key, val in counts_int.items()}
final_distribution_bin = {key: val / shots for key, val in counts_bin.items()}
print(final_distribution_int)

{20: 0.0294, 0: 0.0296, 4: 0.0326, 18: 0.0344, 30: 0.0275, 11: 0.0272, 2: 0.0314, 3: 0.0334, 19: 0.0321, 17: 0.0327, 29: 0.0331, 14: 0.0309, 1: 0.0333, 5: 0.0305, 9: 0.0282, 21: 0.0296, 25: 0.0322, 7: 0.0314, 28: 0.0309, 27: 0.0318, 12: 0.03, 24: 0.0373, 22: 0.0295, 26: 0.0323, 23: 0.0302, 10: 0.0315, 15: 0.0302, 31: 0.0301, 6: 0.0312, 8: 0.0312, 16: 0.0381, 13: 0.0262}


In [13]:
for x in range(2**n):
    if x not in final_distribution_int.keys():
        final_distribution_int[x] = 0

y = np.array([sqrt(final_distribution_int[x]) for x in range(2**n)])

print(y)
print(sqrt(sum(y * y)))

[0.17204651 0.18248288 0.17720045 0.18275667 0.1805547  0.17464249
 0.17663522 0.17720045 0.17663522 0.16792856 0.17748239 0.16492423
 0.17320508 0.16186414 0.17578396 0.17378147 0.19519221 0.18083141
 0.18547237 0.17916473 0.17146428 0.17204651 0.17175564 0.17378147
 0.19313208 0.17944358 0.17972201 0.17832555 0.17578396 0.18193405
 0.16583124 0.17349352]
0.9999999999999999


In [14]:
x_guess = np.array([sqrt(2.0) * y[i] for i in range(N)])
x_guess

array([0.2433105 , 0.25806976, 0.25059928, 0.25845696, 0.25534291,
       0.24698178, 0.24979992, 0.25059928])

In [15]:
temp = (A1 @ x_guess.reshape(-1, 1)).reshape(1, N)

d = 0

for i in range(N):
    if temp[0, i] != 0:
        d += b1[i] / temp[0, i]

d /= N

print(d)

print(1/d * b1)
print(temp)

(0.06165463858332633+0j)
[ 0.        +0.j  0.        +0.j  0.        +0.j  0.        +0.j
  0.33109242+0.j 11.2277475 +0.j  0.92637735+0.j 11.66354021+0.j]
[[3.25893992+0.j 2.99406459+0.j 2.98462745+0.j 2.84249129+0.j
  3.12867701+0.j 3.11277606+0.j 2.98460093+0.j 2.93289529+0.j]]


In [16]:
x = c * d * x_guess

print(x)

[0.01915939+0.j 0.0203216 +0.j 0.01973334+0.j 0.02035209+0.j
 0.02010688+0.j 0.01944848+0.j 0.0196704 +0.j 0.01973334+0.j]


In [17]:
A1 @ x.reshape(-1, 1)

array([[0.25662394+0.j],
       [0.23576644+0.j],
       [0.23502332+0.j],
       [0.22383086+0.j],
       [0.24636644+0.j],
       [0.24511433+0.j],
       [0.23502123+0.j],
       [0.23094969+0.j]])

Kao i u algoritmu $AQC(p)$, i ovde je aproksimacija veoma loša. Stvari se mogu popraviti ako se estimatoru da veći broj pokušaja i ako se broj $M$ poveća. Nažalost, već za $1000$ pokušaja estimatora i $M = 5$, algoritam radi izuzetno sporo na ovom računaru, a pristup pravom kvantnom računaru nemamo.